# เตรียมชื่อลิสต์ไฟล์รายชื่อเรดาร์โมเสคและฝนสถานีรายชั่วโมง
* ต้องการผลลัพธ์ชื่อไฟล์จับคู่ระหว่างฝนเรดาร์กับฝนสถานี เพื่อตรวจสอบค่าความถูกต้องหลังการโมเสคทั้งประเทศ

In [8]:
'''
2024.09.22
โค้ดนี้พัฒนาโดย รองศาสตราจารย์ ดร. นัฐพล มหาวิค ภาควิชาทรัพยากรธรรมชาติและสิ่งแวดล้อม คณะเกษตรศาสตร์ฯ มหาวิทยาลัยนเรศวร 
ในงานวิจัย เรื่อง "การวิจัยและพัฒนาผลิตภัณฑ์โมเสคฝนประมาณค่าจากเรดาร์ตรวจอากาศในพื้นที่ระดับลุ่มน้ำของประเทศไทยด้วยเทคโนโลยีภูมิสารสนเทศรหัสเปิด"
สนับสนุนทุนวิจัยโดยสํานักงานการวิจัยแห่งชาติ (วช.)  แผนงานการวิจัยและนวัตกรรมแผนงานด้านการบริหารจัดการภัยพิบัติทางธรรมชาติ 
ประจำปีงบประมาณ 2566  ตามสัญญา เลขที่ N25A660467 ผู้นำโค้ดนี้ไปใช้หรือดัดแปลงควรอ้างอิงงานวิจัยชิ้นนี้ตามหลักเกณฑ์การอ้างอิงสากล
เรียนหลักการเรดาร์และภูมิสารสนเทศ ที่ https://www.youtube.com/@Nattapon_Mahavik/playlists
หนังสือเรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/144567
หนังสือออนไลน์เรดาร์ตรวจอากาศทางอุตุนิยมวิทยา สำนักพิมพ์จุฬาฯ : https://www.chulabook.com/education/205129
ติดต่อ nattaponm@nu.ac.th
'''
import os
import datetime
import pandas as pd
import time
def ensure_dir(directory):
    os.makedirs(directory, exist_ok=True)

def get_file_datetime(filename, is_radar=True):
    # Extract datetime from filename
    if is_radar:
        date_str = filename.split('.')[0]  # Remove the .tif extension
    else:
        date_str = filename.split('.')[0]
    return datetime.datetime.strptime(date_str, '%Y%m%d%H')

def get_matching_files(radar_dir, gauge_dir):
    radar_files = os.listdir(radar_dir)
    gauge_files = os.listdir(gauge_dir)
    
    radar_times = {get_file_datetime(f, is_radar=True): f for f in radar_files if f.endswith('.tif')}
    gauge_times = {get_file_datetime(f, is_radar=False): f for f in gauge_files if f.endswith('.csv')}
    
    matched_times = sorted(set(radar_times.keys()) & set(gauge_times.keys()))
    
    return [(t, radar_times[t], gauge_times[t]) for t in matched_times]

def save_matched_list(matched_list, output_dir):
    df = pd.DataFrame(matched_list, columns=['Datetime', 'Radar_File', 'Gauge_File'])
    output_file = os.path.join(output_dir, 'matched_rainfall_files.csv')
    
    # Ensure the directory exists
    os.makedirs(os.path.dirname(output_file), exist_ok=True)
    
    # Save the file, overwriting if it already exists
    df.to_csv(output_file, index=False, mode='w')
    
    print(f"Matched list saved to {output_file}")
    return df, output_file

def main():
    radar_dir = r'../00run_batch_acchr_codes/2output/0Hourly/0Sontihn_dbz_single/0mosaic/1mean_mos/0mosaics/0mfb/'
    gauge_dir = r'./0Zprocessing_data/0hourly_rain_thai_mos/0Sontihn/0Hourly/'
    # In your main function:
    output_dir = './0ZProcessing_data/0matched_files_mos_thai/'    
    
    ensure_dir(output_dir)

    #full_path = os.path.abspath(output_dir)
    #print(f"Attempting to access directory: {full_path}")

    # Then try to read the CSV
    #df = pd.read_csv(os.path.join(full_path, 'matched_rainfall_files.csv'))
    
    print("Radar directory contents:")
    print(os.listdir(radar_dir)[:5])  # Print first 5 files
    print("\nGauge directory contents:")
    print(os.listdir(gauge_dir)[:5])  # Print first 5 files
    
    matched_list = get_matching_files(radar_dir, gauge_dir)
    df, output_file = save_matched_list(matched_list, output_dir)
    
    print(f"Total matched times: {len(matched_list)}")
    
    # Add a small delay
    time.sleep(1)
    
    # Print the top 10 rows directly from the DataFrame we just created
    print("\nTop 10 rows of the matched rainfall files:")
    print(df.head(10))
    
    # If you still need to read the file, use the exact path:
    try:
        df_read = pd.read_csv(output_file)
        print("\nSuccessfully read the saved file.")
        print(df_read.head(10))
    except PermissionError as e:
        print(f"Permission error when trying to read the file: {e}")
        print(f"File path attempted: {output_file}")

if __name__ == "__main__":
    main()

Radar directory contents:
['2018071507.tif', '2018071508.tif', '2018071509.tif', '2018071510.tif', '2018071511.tif']

Gauge directory contents:
['2018071500.csv', '2018071501.csv', '2018071502.csv', '2018071503.csv', '2018071504.csv']
Matched list saved to ./0ZProcessing_data/0matched_files_mos_thai/matched_rainfall_files.csv
Total matched times: 240

Top 10 rows of the matched rainfall files:
             Datetime      Radar_File      Gauge_File
0 2018-07-15 07:00:00  2018071507.tif  2018071507.csv
1 2018-07-15 08:00:00  2018071508.tif  2018071508.csv
2 2018-07-15 09:00:00  2018071509.tif  2018071509.csv
3 2018-07-15 10:00:00  2018071510.tif  2018071510.csv
4 2018-07-15 11:00:00  2018071511.tif  2018071511.csv
5 2018-07-15 12:00:00  2018071512.tif  2018071512.csv
6 2018-07-15 13:00:00  2018071513.tif  2018071513.csv
7 2018-07-15 14:00:00  2018071514.tif  2018071514.csv
8 2018-07-15 15:00:00  2018071515.tif  2018071515.csv
9 2018-07-15 16:00:00  2018071516.tif  2018071516.csv

Successf